In [57]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pysal as ps
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
import pickle
from sklearn.svm import SVC
%matplotlib inline
%config IPCompleter.greedy=True


In [34]:
df = pd.read_csv('fent_county_year.txt', sep = '\t')
fent_all_years = pd.read_csv('all_fentanyl_by_county.txt', sep = '\t')
fent_13_17 = pd.read_csv('all_fent_by_county_13-17.txt', sep = '\t')
national_fent = pd.read_csv('national_fent.txt', sep = '\t')
national_all_opioids = pd.read_csv('grouped_by_year_not_county_all_opioids.txt', sep = '\t')
national_fent_cocaine = pd.read_csv('fent_and_cocaine.txt', sep = '\t')
fent_by_state = pd.read_csv('fent_state.txt', sep = '\t')

In [35]:
df = df.drop('Notes', axis = 'columns')
df = df.dropna()
df = df.drop('Year Code', axis = 'columns')
df.columns = ['county', 'county_code', 'year', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_all_years = fent_all_years.drop('Notes', axis = 'columns')
fent_all_years = fent_all_years.dropna()
fent_all_years.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_13_17 = fent_13_17.drop('Notes', axis = 'columns')
fent_13_17 = fent_13_17.dropna()
fent_13_17.columns = ['county', 'county_code', 'deaths', 'population', 'crude_rate', 'aa_rate']

national_fent = national_fent.drop('Notes', axis = 'columns')
national_fent = national_fent.dropna()
national_fent = national_fent.drop('Year Code', axis = 'columns')
national_fent.columns = ['year', 'deaths', 'population', 'crude_rate', 'aa_rate']

national_all_opioids = national_all_opioids.drop('Notes', axis = 'columns')
national_all_opioids = national_all_opioids.dropna()
national_all_opioids = national_all_opioids.drop('Year Code', axis = 'columns')
national_all_opioids.columns = ['year', 'deaths', 'population', 'crude_rate']

national_fent_cocaine = national_fent_cocaine.drop('Notes', axis = 'columns')
national_fent_cocaine = national_fent_cocaine.dropna()
national_fent_cocaine = national_fent_cocaine.drop('Year Code', axis = 'columns')
national_fent_cocaine.columns = ['year', 'deaths', 'population', 'crude_rate', 'aa_rate']

fent_by_state = fent_by_state.drop(['Notes', 'Year Code'], axis = 'columns')
fent_by_state = fent_by_state.dropna()
fent_by_state.columns = ['state', 'state_code', 'year', 'deaths', 'population', 'crude_rate', 'aa_rate']

In [36]:
# dropping hawaii and alaska because they make the map ugly
df['state'] = df.county.apply(lambda x: x[-2:])
df = df[df['state'] != 'AK']
df = df[df['state'] != 'HI']
fent_13_17['state'] = fent_13_17['county'].apply(lambda x: x[-2:])
fent_13_17 = fent_13_17[fent_13_17['state'] != 'AK']
fent_13_17 = fent_13_17[fent_13_17['state'] != 'HI']
fent_by_state_13on = fent_by_state[fent_by_state.year > 2012]
fent_by_state_13on = fent_by_state_13on[fent_by_state_13on.state != 'Alaska']
fent_by_state_13on = fent_by_state_13on[fent_by_state_13on.state != 'Hawaii']

In [37]:
# for now put suppressed entries to 0
def convert_suppressed(entry):
    if entry == 'Suppressed':
        return 0
    elif entry == 'Missing':
        return 0
    else:
        return int(entry)
    
df['deaths'] = df['deaths'].apply(convert_suppressed)
fent_all_years['deaths'] = fent_all_years['deaths'].apply(convert_suppressed)
fent_13_17['deaths'] = fent_13_17['deaths'].apply(convert_suppressed)
national_fent['deaths'] = national_fent['deaths'].apply(convert_suppressed)
national_all_opioids['deaths'] = national_all_opioids['deaths'].apply(convert_suppressed)
national_fent_cocaine['deaths'] = national_fent_cocaine['deaths'].apply(convert_suppressed)
fent_by_state['deaths'] = fent_by_state['deaths'].apply(convert_suppressed)

In [38]:
hdf = df[df.year == 2017][['county', 'county_code', 'state']]
hdf['deaths13'] = list(df[df.year == 2013].deaths)
hdf['deaths14'] = list(df[df.year == 2014].deaths)
hdf['deaths15'] = list(df[df.year == 2015].deaths)
hdf['deaths16'] = list(df[df.year == 2016].deaths)
hdf['deaths17'] = list(df[df.year == 2017].deaths)
hdf['pop13'] = list(df[df.year == 2013].population)
hdf['pop14'] = list(df[df.year == 2014].population)
hdf['pop15'] = list(df[df.year == 2015].population)
hdf['pop16'] = list(df[df.year == 2016].population)
hdf['pop17'] = list(df[df.year == 2017].population)
hdf.reset_index(drop = True, inplace = True)

In [39]:
hdf['total_deaths'] = hdf.merge(fent_13_17[['county', 'deaths']], on = 'county', how = 'left').deaths

In [40]:
counties = gpd.read_file('us-counties.json')

In [41]:
counties.columns = ['county_code', 'name', 'geometry']
counties.county_code = counties.county_code.apply(int)
df.county_code = df.county_code.apply(int)
def convert_suppressed_rate(entry):
    if entry == 'Suppressed':
        return 0
    elif entry == 'Missing':
        return 0
    elif entry == 'Unreliable':
        return 0
    else:
        return float(entry)
    
def calculate_rate(row):
    try:
        deaths = float(row.deaths)
    except ValueError:
        deaths = 0
    try:
        population = float(row.population)
    except ValueError:
        population = 1
    return deaths/population * 100000
    
df['crude_rate'] = df['crude_rate'].apply(convert_suppressed_rate)
df['aa_rate'] = df['aa_rate'].apply(convert_suppressed_rate)
df['calculated_crude_rate'] = df.apply(calculate_rate, axis = 'columns')
geodf = counties[['county_code', 'geometry']].merge(hdf, on = 'county_code')

In [42]:
geodf.pop13 = geodf.pop13.apply(int)
geodf.pop14 = geodf.pop14.apply(int)
geodf.pop15 = geodf.pop15.apply(int)
geodf.pop16 = geodf.pop16.apply(int)
geodf.pop17 = geodf.pop17.apply(int)

In [43]:
rent_burden = pd.read_csv('ACS_17_5YR_B25070_with_ann.csv')
rent_burden = rent_burden[['GEO.id2', 'GEO.display-label', 'HD01_VD01', 'HD01_VD10']]
rent_burden.columns = ['county_code', 'county', 'total_renters', 'burden_over_50_percent']
rent_burden['percent_in_distress'] = rent_burden['burden_over_50_percent']/rent_burden['total_renters']
rent_burden = rent_burden[['county_code', 'percent_in_distress']]
geodf = geodf.merge(rent_burden, on = 'county_code')

In [44]:
prescriptions_17 = pd.read_csv('2017_opioid_prescriptions.csv')[['State/County FIPS Code', '2017']]
prescriptions_16 = pd.read_csv('2016_opioid_prescriptions.csv')[['FIPS County Code', '2016 Prescribing Rate']]
prescriptions_15 = pd.read_csv('2015_opioid_prescriptions.csv')[['FIPS County Code', '2015 Prescribing Rate']]
prescriptions_14 = pd.read_csv('2014_opioid_prescriptions.csv')[['FIPS County Code', '2014 Prescribing Rate']]
prescriptions_13 = pd.read_csv('2013_opioid_prescriptions.csv')[['FIPS County Code', '2013 Prescribing Rate']]
prescriptions_17.columns = ['county_code', 'prate_17']
prescriptions_16.columns = ['county_code', 'prate_16']
prescriptions_15.columns = ['county_code', 'prate_15']
prescriptions_14.columns = ['county_code', 'prate_14']
prescriptions_13.columns = ['county_code', 'prate_13']

In [45]:
geodf = geodf.merge(prescriptions_17, on = 'county_code', how = 'left')
geodf = geodf.merge(prescriptions_16, on = 'county_code', how = 'left')
geodf = geodf.merge(prescriptions_15, on = 'county_code', how = 'left')
geodf = geodf.merge(prescriptions_14, on = 'county_code', how = 'left')
geodf = geodf.merge(prescriptions_13, on = 'county_code', how = 'left')

In [46]:
geodf

,county_code,geometry,county,state,deaths13,deaths14,deaths15,deaths16,deaths17,pop13,...,pop15,pop16,pop17,total_deaths,percent_in_distress,prate_17,prate_16,prate_15,prate_14,prate_13
0,1001,"POLYGON ((-86.41178600000001 32.706342, -86.41...","Autauga County, AL",AL,0,0,0,0,0,55246,...,55347,55416,55504,0,0.262802,106.6,129.6,129.9,145.3,166.7
1,1003,"POLYGON ((-87.76459 31.298768, -87.616713 31.2...","Baldwin County, AL",AL,0,0,0,0,0,195540,...,203709,208563,212628,20,0.209795,106.7,123.8,132.1,143.5,154.3
2,1005,"POLYGON ((-85.354736 32.147694, -85.053504 32....","Barbour County, AL",AL,0,0,0,0,0,27076,...,26489,25965,25270,0,0.210969,90.7,92.7,93.3,102,107.5
3,1007,"POLYGON ((-87.063542 33.248559, -87.025203 33....","Bibb County, AL",AL,0,0,0,0,0,22512,...,22583,22643,22668,0,0.200704,80.6,97.2,69.4,75.8,70.5
4,1009,"POLYGON ((-86.488463 34.261793, -86.455601 34....","Blount County, AL",AL,0,0,0,0,0,57872,...,57673,57704,58013,13,0.133438,48.9,56.9,57.9,63.2,65.9
5,1011,"POLYGON ((-85.91886100000001 32.273663, -85.43...","Bullock County, AL",AL,0,0,0,0,0,10639,...,10696,10362,10309,0,0.379091,19.9,19.8,23.4,34.3,41.2
6,1013,"POLYGON ((-86.477509 31.966955, -86.450124 31....","Butler County, AL",AL,0,0,0,0,0,20265,...,20154,19998,19825,0,0.143602,118.6,135.4,140.5,145,149.7
7,1015,"POLYGON ((-85.738122 33.966038, -85.5299980000...","Calhoun County, AL",AL,0,0,0,0,0,116736,...,115620,114611,114728,0,0.211533,138.2,161,165.4,180,188.9
8,1017,"POLYGON ((-85.40402899999999 33.106158, -85.23...","Chambers County, AL",AL,0,0,0,0,0,34162,...,34123,33843,33713,0,0.161838,127.0,140.6,147.3,163.1,169.7
9,1019,"POLYGON ((-85.51356699999999 34.524686, -85.46...","Cherokee County, AL",AL,0,0,0,0,0,26203,...,25859,25725,25857,0,0.187712,128.3,157.7,169.2,175.1,163.8


In [47]:
def convert_prate(value):
    try:
        return float(value)
    except ValueError:
        return None
geodf['prate_17'] = geodf.prate_17.apply(convert_prate)
geodf['prate_16'] = geodf.prate_16.apply(convert_prate)
geodf['prate_15'] = geodf.prate_15.apply(convert_prate)
geodf['prate_14'] = geodf.prate_14.apply(convert_prate)
geodf['prate_13'] = geodf.prate_13.apply(convert_prate)

In [48]:
w17 = ps.queen_from_shapefile('geodf2017.shp')
w16 = ps.queen_from_shapefile('geodf2016.shp')
w15 = ps.queen_from_shapefile('geodf2015.shp')
w14 = ps.queen_from_shapefile('geodf2014.shp')
w13 = ps.queen_from_shapefile('geodf2013.shp')

/home/steve/anaconda3/lib/python3.7/site-packages/pysal/core/IOHandlers/pyShpIO.py:172: RuntimeWarning: SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
  warn("SHAPEFILE WARNING: Polygon %d topology has been fixed. (ccw -> cw)" % (self.pos), RuntimeWarning)
/home/steve/anaconda3/lib/python3.7/site-packages/pysal/core/IOHandlers/pyShpIO.py:172: RuntimeWarning: SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
  warn("SHAPEFILE WARNING: Polygon %d topology has been fixed. (ccw -> cw)" % (self.pos), RuntimeWarning)


SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 195 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 2880 topology has been fixed. (ccw -> cw)
SHAPEFILE WARNING: Polygon 195 topology

In [49]:
# row standardization 
w17.transform = 'r'
w16.transform = 'r'
w15.transform = 'r'
w14.transform = 'r'
w13.transform = 'r'

In [50]:
geodf['crude_rate13'] = geodf.deaths13 / geodf.pop13 * 100000
geodf['crude_rate14'] = geodf.deaths14 / geodf.pop14 * 100000
geodf['crude_rate15'] = geodf.deaths15 / geodf.pop15 * 100000
geodf['crude_rate16'] = geodf.deaths16 / geodf.pop16 * 100000
geodf['crude_rate17'] = geodf.deaths17 / geodf.pop17 * 100000

In [51]:
geodf['crude_rate_lag13'] = ps.lag_spatial(w13, geodf['crude_rate13'])
geodf['crude_rate_lag14'] = ps.lag_spatial(w14, geodf['crude_rate14'])
geodf['crude_rate_lag15'] = ps.lag_spatial(w15, geodf['crude_rate15'])
geodf['crude_rate_lag16'] = ps.lag_spatial(w16, geodf['crude_rate16'])
geodf['crude_rate_lag17'] = ps.lag_spatial(w17, geodf['crude_rate17'])

In [52]:
# new label, 1 if the following year as greater than 0 deaths, 0 otherwise
def assign_label(entry):
    if entry > 0:
        return 1
    else:
        return 0
geodf['label_17'] = geodf['crude_rate17'].apply(assign_label)
geodf['label_16'] = geodf['crude_rate16'].apply(assign_label)
geodf['label_15'] = geodf['crude_rate15'].apply(assign_label)

In [53]:
only_zeroes = geodf[(geodf.deaths13 + geodf.deaths14 + geodf.deaths15 + geodf.deaths16) == 0]

In [54]:
only_zeroes['change_label'] = only_zeroes.deaths17.apply(assign_label)

/home/steve/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
only_zeroes

,county_code,geometry,county,state,deaths13,deaths14,deaths15,deaths16,deaths17,pop13,...,crude_rate17,crude_rate_lag13,crude_rate_lag14,crude_rate_lag15,crude_rate_lag16,crude_rate_lag17,label_17,label_16,label_15,change_label
0,1001,"POLYGON ((-86.41178600000001 32.706342, -86.41...","Autauga County, AL",AL,0,0,0,0,0,55246,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
1,1003,"POLYGON ((-87.76459 31.298768, -87.616713 31.2...","Baldwin County, AL",AL,0,0,0,0,0,195540,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.903740,0,0,0,0
2,1005,"POLYGON ((-85.354736 32.147694, -85.053504 32....","Barbour County, AL",AL,0,0,0,0,0,27076,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
3,1007,"POLYGON ((-87.063542 33.248559, -87.025203 33....","Bibb County, AL",AL,0,0,0,0,0,22512,...,0.000000,0.000000,0.580111,0.959063,3.692094,3.055752,0,0,0,0
4,1009,"POLYGON ((-86.488463 34.261793, -86.455601 34....","Blount County, AL",AL,0,0,0,0,0,57872,...,0.000000,0.000000,0.580111,0.959063,2.426003,4.708467,0,0,0,0
5,1011,"POLYGON ((-85.91886100000001 32.273663, -85.43...","Bullock County, AL",AL,0,0,0,0,0,10639,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
6,1013,"POLYGON ((-86.477509 31.966955, -86.450124 31....","Butler County, AL",AL,0,0,0,0,0,20265,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
7,1015,"POLYGON ((-85.738122 33.966038, -85.5299980000...","Calhoun County, AL",AL,0,0,0,0,0,116736,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.919566,0,0,0,0
8,1017,"POLYGON ((-85.40402899999999 33.106158, -85.23...","Chambers County, AL",AL,0,0,0,0,0,34162,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0
9,1019,"POLYGON ((-85.51356699999999 34.524686, -85.46...","Cherokee County, AL",AL,0,0,0,0,0,26203,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.085404,0,0,0,0


In [56]:
# test with a lot of variables, accuracy as measure, baseline of all zeroes is 94.85
independent_variables17 = only_zeroes[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'pop13', 'pop14', 'pop15',
                                 'pop16', 'percent_in_distress', 'prate_16', 'prate_15', 'prate_14', 'prate_13',
                                  'crude_rate13', 'crude_rate14', 'crude_rate15', 'crude_rate16', 'crude_rate_lag13',
                                 'crude_rate_lag14', 'crude_rate_lag15', 'crude_rate_lag16']]
labels = only_zeroes['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

accuracy:  0.9503676470588235
[[509   7]
 [ 20   8]]
recall:  0.2857142857142857
accuracy:  0.9430147058823529
[[504  12]
 [ 19   9]]
recall:  0.32142857142857145
accuracy:  0.9393382352941176
[[507   9]
 [ 24   4]]
recall:  0.14285714285714285
accuracy:  0.9558823529411765
[[510   6]
 [ 18  10]]
recall:  0.35714285714285715
accuracy:  0.9485294117647058
[[507   9]
 [ 19   9]]
recall:  0.32142857142857145
accuracy:  0.9522058823529411
[[511   5]
 [ 21   7]]
recall:  0.25
accuracy:  0.9503676470588235
[[511   5]
 [ 22   6]]
recall:  0.21428571428571427
accuracy:  0.9466911764705882
[[509   7]
 [ 22   6]]
recall:  0.21428571428571427
accuracy:  0.9466911764705882
[[507   9]
 [ 20   8]]
recall:  0.2857142857142857
accuracy:  0.9540441176470589
[[511   5]
 [ 20   8]]
recall:  0.2857142857142857


baseline accuracy:  94.85
accuracy mean:  0.9487132352941178
recall mean:  0.26785714285714285


In [ ]:
# not a good model because it has a low recall

In [60]:
# test with a lot of variables, accuracy as measure, baseline of all zeroes is 94.85
independent_variables17 = only_zeroes.dropna()[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'pop13', 'pop14', 'pop15',
                                 'pop16', 'percent_in_distress', 'prate_16', 'prate_15', 'prate_14', 'prate_13',
                                  'crude_rate13', 'crude_rate14', 'crude_rate15', 'crude_rate16', 'crude_rate_lag13',
                                 'crude_rate_lag14', 'crude_rate_lag15', 'crude_rate_lag16']]
labels = only_zeroes.dropna()['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = SVC()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0
accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0


/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0
accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0


/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0
accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0


/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0
accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0


/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/steve/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


accuracy:  0.9402985074626866
[[441   0]
 [ 28   0]]
recall:  0.0


baseline accuracy:  94.85
accuracy mean:  0.9402985074626867
recall mean:  0.0


In [62]:
# test with fewer variables, accuracy as measure
independent_variables17 = only_zeroes[['pop16', 'percent_in_distress', 'prate_16', 'crude_rate_lag16']]
labels = only_zeroes['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

accuracy:  0.9485294117647058
[[508   8]
 [ 20   8]]
recall:  0.2857142857142857
accuracy:  0.9411764705882353
[[506  10]
 [ 22   6]]
recall:  0.21428571428571427
accuracy:  0.9466911764705882
[[510   6]
 [ 23   5]]
recall:  0.17857142857142858
accuracy:  0.9466911764705882
[[510   6]
 [ 23   5]]
recall:  0.17857142857142858
accuracy:  0.9522058823529411
[[513   3]
 [ 23   5]]
recall:  0.17857142857142858
accuracy:  0.9411764705882353
[[502  14]
 [ 18  10]]
recall:  0.35714285714285715
accuracy:  0.9540441176470589
[[512   4]
 [ 21   7]]
recall:  0.25
accuracy:  0.9448529411764706
[[509   7]
 [ 23   5]]
recall:  0.17857142857142858
accuracy:  0.9503676470588235
[[509   7]
 [ 20   8]]
recall:  0.2857142857142857
accuracy:  0.9522058823529411
[[511   5]
 [ 21   7]]
recall:  0.25


baseline accuracy:  94.85
accuracy mean:  0.9477941176470587
recall mean:  0.2357142857142857


In [102]:
# has less variables but has lower accuracy and recall

In [130]:
# try to get model to have better recall, probably at the cost of accuracy
independent_variables17 = only_zeroes[['deaths13', 'deaths14', 'deaths15', 'deaths16', 'pop13', 'pop14', 'pop15',
                                 'pop16', 'percent_in_distress', 'prate_16', 'prate_15', 'prate_14', 'prate_13',
                                  'crude_rate13', 'crude_rate14', 'crude_rate15', 'crude_rate16', 'crude_rate_lag13',
                                 'crude_rate_lag14', 'crude_rate_lag15', 'crude_rate_lag16']]
labels = only_zeroes['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier(scale_pos_weight = (len(labels) - sum(labels)) / sum(labels))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

accuracy:  0.90625
[[471  45]
 [  6  22]]
recall:  0.7857142857142857
accuracy:  0.9025735294117647
[[468  48]
 [  5  23]]
recall:  0.8214285714285714
accuracy:  0.8805147058823529
[[461  55]
 [ 10  18]]
recall:  0.6428571428571429
accuracy:  0.8897058823529411
[[459  57]
 [  3  25]]
recall:  0.8928571428571429
accuracy:  0.8970588235294118
[[467  49]
 [  7  21]]
recall:  0.75
accuracy:  0.8823529411764706
[[459  57]
 [  7  21]]
recall:  0.75
accuracy:  0.8805147058823529
[[458  58]
 [  7  21]]
recall:  0.75
accuracy:  0.8988970588235294
[[466  50]
 [  5  23]]
recall:  0.8214285714285714
accuracy:  0.8988970588235294
[[466  50]
 [  5  23]]
recall:  0.8214285714285714
accuracy:  0.8731617647058824
[[456  60]
 [  9  19]]
recall:  0.6785714285714286


baseline accuracy:  94.85
accuracy mean:  0.8909926470588235
recall mean:  0.7714285714285715


In [ ]:
# it worked perfectly! the recall is much higher and the accuracy barely got hurt

In [135]:
# same as above, going for higher recall but this time with smaller list of variables
independent_variables17 = only_zeroes[['deaths16','pop16', 'percent_in_distress', 'prate_16', 'crude_rate16', 'crude_rate_lag16']]
labels = only_zeroes['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier(scale_pos_weight = (len(labels) - sum(labels)) / sum(labels))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

accuracy:  0.8823529411764706
[[463  53]
 [ 11  17]]
recall:  0.6071428571428571
accuracy:  0.9044117647058824
[[468  48]
 [  4  24]]
recall:  0.8571428571428571
accuracy:  0.8805147058823529
[[459  57]
 [  8  20]]
recall:  0.7142857142857143
accuracy:  0.8878676470588235
[[460  56]
 [  5  23]]
recall:  0.8214285714285714
accuracy:  0.8529411764705882
[[441  75]
 [  5  23]]
recall:  0.8214285714285714
accuracy:  0.8529411764705882
[[446  70]
 [ 10  18]]
recall:  0.6428571428571429
accuracy:  0.8713235294117647
[[450  66]
 [  4  24]]
recall:  0.8571428571428571
accuracy:  0.8676470588235294
[[454  62]
 [ 10  18]]
recall:  0.6428571428571429
accuracy:  0.875
[[455  61]
 [  7  21]]
recall:  0.75
accuracy:  0.8602941176470589
[[451  65]
 [ 11  17]]
recall:  0.6071428571428571


baseline accuracy:  94.85
accuracy mean:  0.8735294117647058
recall mean:  0.7321428571428571


In [64]:
# same as above, going for higher recall but this time with smaller list of variables
independent_variables17 = only_zeroes[['pop16', 'percent_in_distress', 'prate_16', 'crude_rate_lag16']]
labels = only_zeroes['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier(scale_pos_weight = (len(labels) - sum(labels)) / sum(labels))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

accuracy:  0.9172794117647058
[[475  41]
 [  4  24]]
recall:  0.8571428571428571
accuracy:  0.8805147058823529
[[455  61]
 [  4  24]]
recall:  0.8571428571428571
accuracy:  0.8676470588235294
[[451  65]
 [  7  21]]
recall:  0.75
accuracy:  0.8786764705882353
[[456  60]
 [  6  22]]
recall:  0.7857142857142857
accuracy:  0.8915441176470589
[[464  52]
 [  7  21]]
recall:  0.75
accuracy:  0.8805147058823529
[[458  58]
 [  7  21]]
recall:  0.75
accuracy:  0.8915441176470589
[[465  51]
 [  8  20]]
recall:  0.7142857142857143
accuracy:  0.8952205882352942
[[461  55]
 [  2  26]]
recall:  0.9285714285714286
accuracy:  0.9136029411764706
[[473  43]
 [  4  24]]
recall:  0.8571428571428571
accuracy:  0.8823529411764706
[[456  60]
 [  4  24]]
recall:  0.8571428571428571


baseline accuracy:  94.85
accuracy mean:  0.8898897058823529
recall mean:  0.8107142857142856


In [ ]:
# small sacrificies on recall and mean but I think it's worth it to have a 
# smaller amount of variables to input on the website

In [137]:
# trying without crude_rate_lag to see how important it is
independent_variables17 = only_zeroes[['deaths16','pop16', 'percent_in_distress', 'prate_16', 'crude_rate16']]
labels = only_zeroes['label_17']
accuracy_list = []
recall_list = []
for i in range(10):
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables17, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier(scale_pos_weight = (len(labels) - sum(labels)) / sum(labels))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list.append(acc_score)
    recall_list.append(rec_score)
    print('accuracy: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall: ', rec_score)
print('')
print('')
print('baseline accuracy: ', 94.85)
print('accuracy mean: ', np.mean(accuracy_list))
print('recall mean: ', np.mean(recall_list))

accuracy:  0.8290441176470589
[[429  87]
 [  6  22]]
recall:  0.7857142857142857
accuracy:  0.8308823529411765
[[426  90]
 [  2  26]]
recall:  0.9285714285714286
accuracy:  0.8143382352941176
[[422  94]
 [  7  21]]
recall:  0.75
accuracy:  0.84375
[[443  73]
 [ 12  16]]
recall:  0.5714285714285714
accuracy:  0.8474264705882353
[[440  76]
 [  7  21]]
recall:  0.75
accuracy:  0.8069852941176471
[[420  96]
 [  9  19]]
recall:  0.6785714285714286
accuracy:  0.8198529411764706
[[424  92]
 [  6  22]]
recall:  0.7857142857142857
accuracy:  0.8235294117647058
[[429  87]
 [  9  19]]
recall:  0.6785714285714286
accuracy:  0.8400735294117647
[[438  78]
 [  9  19]]
recall:  0.6785714285714286
accuracy:  0.8308823529411765
[[430  86]
 [  6  22]]
recall:  0.7857142857142857


baseline accuracy:  94.85
accuracy mean:  0.8286764705882353
recall mean:  0.7392857142857143


In [ ]:
# seems important I'll keep it in

In [65]:
# making sure that models will generalize to two years in advance
sixteen_zeroes = geodf[(geodf.deaths13 + geodf.deaths14 + geodf.deaths15) == 0]
independent_variables16 = sixteen_zeroes[['pop15', 'percent_in_distress', 'prate_15', 'crude_rate_lag15']]
accuracy_list_16 = []
recall_list_16 = []
accuracy_list_17 = []
recall_list_17 = []
for i in range(10):
    labels = sixteen_zeroes['label_16']
    X_train, X_test, Y_train, Y_test = train_test_split(independent_variables16, labels, test_size = 0.2, stratify = labels)
    model = XGBClassifier(scale_pos_weight = (len(labels) - sum(labels)) / sum(labels))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(Y_test, y_pred)
    rec_score = recall_score(Y_test, y_pred)
    accuracy_list_16.append(acc_score)
    recall_list_16.append(rec_score)
    print('accuracy on 2016: ', acc_score)
    print(confusion_matrix(Y_test, y_pred))
    print('recall on 2016: ', rec_score)
    print('')
    print('')
    labels = only_zeroes['label_17']
    predict_df = only_zeroes[['pop16', 'percent_in_distress', 'prate_16', 'crude_rate_lag16']].copy()
    predict_df.columns = ['pop15', 'percent_in_distress', 'prate_15', 'crude_rate_lag15']
    pred_17 = model.predict(predict_df)
    acc_score = accuracy_score(labels, pred_17)
    rec_score = recall_score(labels, pred_17)
    accuracy_list_17.append(acc_score)
    recall_list_17.append(rec_score)
    print('accuracy on 2017: ', acc_score)
    print(confusion_matrix(labels, pred_17))
    print('recall on 2017: ', rec_score)
    print('')
print('')
print('--------------------------------------------------------------------')
print('')
print('accuracy mean 2016: ', np.mean(accuracy_list_16))
print('recall mean 2016: ', np.mean(recall_list_16))
print('accuracy mean 2017: ', np.mean(accuracy_list_17))
print('recall mean 2017: ', np.mean(recall_list_17))

accuracy on 2016:  0.9035087719298246
[[495  48]
 [  7  20]]
recall on 2016:  0.7407407407407407


accuracy on 2017:  0.882179675994109
[[2297  281]
 [  39   99]]
recall on 2017:  0.717391304347826

accuracy on 2016:  0.868421052631579
[[474  69]
 [  6  21]]
recall on 2016:  0.7777777777777778


accuracy on 2017:  0.8814432989690721
[[2291  287]
 [  35  103]]
recall on 2017:  0.7463768115942029

accuracy on 2016:  0.8859649122807017
[[485  58]
 [  7  20]]
recall on 2016:  0.7407407407407407


accuracy on 2017:  0.8902798232695139
[[2317  261]
 [  37  101]]
recall on 2017:  0.7318840579710145

accuracy on 2016:  0.887719298245614
[[484  59]
 [  5  22]]
recall on 2016:  0.8148148148148148


accuracy on 2017:  0.8766568483063328
[[2287  291]
 [  44   94]]
recall on 2017:  0.6811594202898551

accuracy on 2016:  0.8929824561403509
[[486  57]
 [  4  23]]
recall on 2016:  0.8518518518518519


accuracy on 2017:  0.8836524300441826
[[2301  277]
 [  39   99]]
recall on 2017:  0.717391304347826



In [ ]:
# great! results from the 2015 model trained to predict 2016 are still good on 2017, meaning our 2016 model should
# have a reasonable shot at predicting 2018


In [67]:
# training the final model for all values, still 2016 - 2017
# same as above, going for higher recall but this time with smaller list of variables
independent_variables17 = only_zeroes[['deaths16','pop16', 'percent_in_distress', 'prate_16', 'crude_rate16', 'crude_rate_lag16']]
labels = only_zeroes['label_17']
model = XGBClassifier(scale_pos_weight = (len(labels) - sum(labels)) / sum(labels))
model.fit(independent_variables17, labels)
predicting_17_df = only_zeroes[['deaths17','pop17', 'percent_in_distress', 'prate_17', 'crude_rate17', 'crude_rate_lag17']].copy()
predicting_17_df.columns = ['deaths16','pop16', 'percent_in_distress', 'prate_16', 'crude_rate16', 'crude_rate_lag16']
only_zeroes['predicted_18'] = model.predict(predicting_17_df)
with open('fentanyl-model.pkl', 'wb') as filename:
    pickle.dump(model, filename)


/home/steve/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [161]:
with open('fentanyl-model.pkl', 'rb') as filename:
    loaded_model = pickle.load(filename)
loaded_predictions = loaded_model.predict(predicting_17_df)
print(len(loaded_predictions))
print(sum(loaded_predictions == only_zeroes['predicted_18']))

2716
2716


In [ ]:
# dumping the model works

In [68]:
only_zeroes.predicted_18.sum()

514

In [ ]:
# enter county name to autofill the boxes 
# display already happening if deaths > 0
# only pass zeroes to the model, so do the check then pass the model

In [151]:
only_zeroes

,county_code,geometry,county,state,deaths13,deaths14,deaths15,deaths16,deaths17,pop13,...,crude_rate_lag13,crude_rate_lag14,crude_rate_lag15,crude_rate_lag16,crude_rate_lag17,label_17,label_16,label_15,change_label,predicted_18
0,1001,"POLYGON ((-86.41178600000001 32.706342, -86.41...","Autauga County, AL",AL,0,0,0,0,0,55246,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
1,1003,"POLYGON ((-87.76459 31.298768, -87.616713 31.2...","Baldwin County, AL",AL,0,0,0,0,0,195540,...,0.000000,0.000000,0.000000,0.000000,0.903740,0,0,0,0,1
2,1005,"POLYGON ((-85.354736 32.147694, -85.053504 32....","Barbour County, AL",AL,0,0,0,0,0,27076,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
3,1007,"POLYGON ((-87.063542 33.248559, -87.025203 33....","Bibb County, AL",AL,0,0,0,0,0,22512,...,0.000000,0.580111,0.959063,3.692094,3.055752,0,0,0,0,0
4,1009,"POLYGON ((-86.488463 34.261793, -86.455601 34....","Blount County, AL",AL,0,0,0,0,0,57872,...,0.000000,0.580111,0.959063,2.426003,4.708467,0,0,0,0,0
5,1011,"POLYGON ((-85.91886100000001 32.273663, -85.43...","Bullock County, AL",AL,0,0,0,0,0,10639,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
6,1013,"POLYGON ((-86.477509 31.966955, -86.450124 31....","Butler County, AL",AL,0,0,0,0,0,20265,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
7,1015,"POLYGON ((-85.738122 33.966038, -85.5299980000...","Calhoun County, AL",AL,0,0,0,0,0,116736,...,0.000000,0.000000,0.000000,0.000000,2.919566,0,0,0,0,1
8,1017,"POLYGON ((-85.40402899999999 33.106158, -85.23...","Chambers County, AL",AL,0,0,0,0,0,34162,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
9,1019,"POLYGON ((-85.51356699999999 34.524686, -85.46...","Cherokee County, AL",AL,0,0,0,0,0,26203,...,0.000000,0.000000,0.000000,0.000000,2.085404,0,0,0,0,0
